In [ ]:
# !pip install plotly matplotlib pandas requests
# !git clone https://github.com/HuobiRDCenter/huobi_Python
# !cd huobi_Python && python setup.py install

In [1]:
# hide it
import datetime
import hashlib
import hmac
import json
import os
import sys
import base64
import pandas as pd
from urllib.parse import urlencode
import requests

if sys.platform == "win32" or sys.platform == "darwin":
    os.environ.update({"HTTP_PROXY": "socks5h://127.0.0.1:12315"})
    os.environ.update({"HTTPS_PROXY": "socks5h://127.0.0.1:12315"})


host = "https://api.hbdm.com"

def send_swap_requests(method: str, url: str, **kwargs) -> dict:

    query = {
        "AccessKeyId": api_key,
        "SignatureMethod": "HmacSHA256",
        "SignatureVersion": "2",
        "Timestamp": datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S")
    }
    if method.lower() == "get":
        query.update(kwargs)
        query = dict(sorted(query.items()))

    query = urlencode(query)
    prepare_sig = method.upper()+"\n"+host[8:]+"\n"+url+"\n"+query
    pre_sig = hmac.new(secret_key.encode("utf8"),
                       prepare_sig.encode("utf8"), 
                       digestmod=hashlib.sha256)
    sig_part = urlencode({"Signature": base64.b64encode(pre_sig.digest()).decode("utf8")})
    query = query+"&"+sig_part
    if method.lower() == "post":
        resp = requests.request(method, host+url+"?"+query, json=kwargs)
    elif method.lower() == "get":
        resp = requests.request(method, host+url+"?"+query)
    return json.loads(resp.content.decode("utf8"))

def get_current_swap_info(contract_code: str) -> dict:
    method = "post"
    url = "/linear-swap-api/v1/swap_position_info"
    resp = send_swap_requests(method, url, contract_code=contract_code)
    if resp.get("data"):
        return resp["data"][0]
    else:
        return None

def fast_close(contract_code: str, volume: int, direction: str):
    method = "POST"
    url = "/linear-swap-api/v1/swap_lightning_close_position"
    resp = send_swap_requests(method, url, contract_code=contract_code, volume=int(volume), direction=direction)
    # print(resp)
    if resp.get("status") != "ok":
        resp = fast_close(contract_code, volume, direction)
    return resp

def get_kline(contract_code: str, period: str="1min", size: str=1):
    method = "get"
    url = "/linear-swap-ex/market/history/kline"
    resp = send_swap_requests(method, url, contract_code=contract_code, period=period, size=size)
    return resp["data"]

def reverse_direction(direction: str):
    if direction == "buy":
        return "sell"
    elif direction == "sell":
        return "buy"
    else:
        Exception("wha????")

def get_price(contract_code, period="1min", size=1):
    raw_candle_data = get_kline(contract_code, period=period, size=size)
    raw_candle_data = raw_candle_data[::-1]
    final_dict = {"date":[], "high":[], "low":[], "open":[], "close":[], "volume":[]}
    for one_data in raw_candle_data:
        if period == "1day":
            final_dict["date"].append(datetime.datetime.utcfromtimestamp(one_data["id"])+datetime.timedelta(hours=8))
        else:
            final_dict["date"].append(datetime.datetime.utcfromtimestamp(one_data["id"])+datetime.timedelta(hours=8))
        final_dict["high"].append(one_data["high"])
        final_dict["low"].append(one_data["low"])
        final_dict["open"].append(one_data["open"])
        final_dict["close"].append(one_data["close"])
        final_dict["volume"].append(one_data["vol"])
    return  pd.DataFrame(final_dict).set_index("date")


In [ ]:
import datetime
import random
from plotly import graph_objects as go
from IPython.display import clear_output
from time import sleep


exec_mode = "real" # real
contract = "ETH-USDT"



######### 从服务器获取当前合约持仓信息 ##########
stime = datetime.datetime.now()
pr = 0
max_rate = 0
plot_list = []

# 简单的平仓策略配置
init_win_rates = 20
force_close = -30
change_rate = 10
if exec_mode == "real":
    while True:
        clear_output(wait=True)
        print("waiting 4 ur order"+"".join(['.' for _ in range(0,random.randint(0,5))])) 
        result = get_current_swap_info(contract)
        print(result)
        if result is not None:
            break
    open_point=result.get('cost_open')
    leverage = result.get('lever_rate')
    direction = result.get('direction')
    volume = result.get("volume")
################################################

######### 随便这玩，注释上面内容，使用下面内容 ###############
elif exec_mode == "test":                              # 
    open_point = get_price(contract).iloc[-1]["close"] # 服务启动则买入
    leverage = 75                                      # 杠杆
    direction = "buy"                                  # 方向
    volume = 1                                         # 手数，没什么用
########################################################

else:
    raise Exception("what?")

while True:
    try:
        current_point = get_price(contract, "1min", 1).iloc[-1]["close"]
    except Exception as ex:
        print("!!!!!!", ex)
        continue
    # 计算当前订单涨跌幅
    if direction in "buy":
        rates = (current_point - open_point) / open_point * 100 * leverage
    elif direction in "sell":
        rates = (open_point - current_point) / open_point * 100 * leverage
    else:
        raise Exception("wha???????")
    if pr != rates:
        clear_output(wait=True)
        if max_rate < rates:
            max_rate = rates
        duration = datetime.datetime.now()-stime
        print(f"It is {duration} later")
        print(f"current contract price {current_point}")
        print("current rate = {:.4f}%".format(rates))
        print("================================")
        print("The max rate: {:.4f}%".format(max_rate))
        print(f"You {direction} {contract} with {open_point}\nThe force close line: {force_close}\nThe win rates: {init_win_rates}")
        plot_list.append(rates)
    pr = rates
    if rates < force_close:
        if rates < 0:
            print("I quite QAQ")
            fast_close(contract, volume, reverse_direction(direction))
        else:
            print(f"I quite XD, with {rates}%")
            fast_close(contract, volume, reverse_direction(direction))
        break
    if rates > 2* init_win_rates:
        print(f"I quite XD, with {rates}%")
        # wallet = wallet * (100+rates)/100
    elif rates > init_win_rates*0.9:
        force_close = init_win_rates - change_rate
        init_win_rates += change_rate
    else:
        print("???????")
# print(f"{wallet} usdt left")

fig = go.Figure(
    go.Scatter(y=plot_list)
)

fig.show()

It is 0:01:02.957738 later
current contract price 3378.25
current rate = 1.6203%
The max rate: 3.9065%
You sell ETH-USDT with 3378.98
The force close line: -30
The win rates: 20
???????
???????
???????
???????
???????
???????


In [ ]:
import datetime
import requests
import pandas as pd
import random
from utils import *
from strategies import *
from matplotlib import pyplot as plt

In [ ]:
symbol_list = get_symbol_list()
sb = symbol_list[random.randint(0, len(symbol_list))]
for sb in symbol_list:
    a=get_price(sb, "30min", 2000)
    a=get_rate(a, "close")
    a["cr+1"] = (a["close"].shift(-1) - a["close"]) / a["close"]
    a["cr+2"] = (a["close"].shift(-2) - a["close"]) / a["close"]
    a=get_rate(a, "volume")
    a=custom_data_analyze_func(a, rolling_days=3)
    standard = a.describe()["vol_mean3_cr"]["75%"]
    c=a.loc[(a["vol_mean3_cr"]>standard) & (a["close_rate"]>0)]
    try:
        if datetime.datetime.strptime("2021-04-22 17:00:00", "%Y-%m-%d %H:%M:%S") == c.index[-1]:
            print(sb)
    except:
        continue
print("done")

In [ ]:
# symbol_list = get_symbol_list()
# sb = symbol_list[random.randint(0, len(symbol_list))]
sb = "mtausdt"
a=get_price(sb, "30min", 2000)
print(sb)
a=get_rate(a, "close")
a["cr+1"] = (a["close"].shift(-1) - a["close"]) / a["close"]
a["cr+2"] = (a["close"].shift(-2) - a["close"]) / a["close"]
a=get_rate(a, "volume")
a=custom_data_analyze_func(a, rolling_days=3)
standard = a.describe()["vol_mean3_cr"]["75%"]
c=a.loc[(a["vol_mean3_cr"]>standard) & (a["close_rate"]>0)]
print(c.index[-1], c.iloc[-1]["close"])
display(c[["cr+1","cr+2","close_rate"]].describe())
# c[["cr+1"]].plot(kind="hist")
aa=get_price(sb, "1min", 2000)
aa=get_mean_line(aa)
bb = aa[c.index[-1]-datetime.timedelta(hours=0.5) : c.index[-1]]
draw_candle(bb)
cc = aa[c.index[-1]:c.index[-1]+datetime.timedelta(hours=0.5)]
draw_candle(cc)